# Gradients

In [1]:
%reset -f

In [2]:
import tensorflow as tf
import torch
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("default")

2024-11-30 14:26:30.703187: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Backward et Foreward





### Introduction

Mais comment calcule-t-on des dérivées déjà ? Ok, on sait le faire sur papier, mais avec un ordinateur ? Curieusement, la bonne réponse est venue tardivement (1985 environ). Cela s'appelle "la rétro-propagation du gradient" (=back-propagation). Et cette avancée algorithmique a permie de développer le deep learning.

Imaginons que nous voulons effectuer le calcul $y=3x*(x^2+ 4)$ et déterminer son gradient $\frac{dy}{dx}=3(x^2+ 4)+3x*2x$ en un point précis: $x=3$. On agit alors en deux étapes:

* Passage forward: On effectue le calcul voulu  tout en enregistrant chaque étape de calcul, ici: $x^2$, puis $x^2+4$, ensuite $3x$, ensuite $y$.  Techniquement en tensorflow, l'enregistrement se fait sur une "bande" (imaginez une bande magnétique) via l'objet `tf.GradientTape`.
* Passage *backward*:  On parcourt cette bande dans l'ordre inverse pour calculer et composer les gradients des opérations élémentaires (on verra un exemple plus loin).  



In [3]:
x = tf.Variable(3.0)

# Passage forward
with tf.GradientTape() as tape:
  y = 3*x*(x**2 + x)

# Passage backward
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

99.0

### Dérivation composée (Chain rule)

Détaillons les maths sur un exemple

Analysons la dérivée de la fonction $  h \circ g \circ f (x)  $.  Voici son graphe de calcul:

$$
x \xrightarrow  f   y   \xrightarrow g z   \xrightarrow h  t
$$
  Les accroissements infinitésimaux se multiplient (c'est la chain rule) :

\begin{alignat}{1}
\frac {\partial  t  }{\partial x}      &=        \frac{ \partial y }{ \partial x}   \frac{ \partial z }{ \partial y}   \frac{ \partial t }{ \partial z } \\
&=  f'(x) g'(y) h'(z)    
\end{alignat}






Nous allons décortiquer le calcul de cette dérivée en un point précis. Pour fixer les idées:

* $f(a) = \sin(a)$, donc $f'(a) = \cos(a)$
* $g(a) =  4 a^2$, donc $g'(a) = 8 a$
* $h(a)=  \tanh(a)$ donc $h'(a)= 1-\tanh^2(a)$

Notons que l'ordinateur sait calculer ces fonctions et leur dérivée de manière très précise.

Nous voulons calculer
$$
   \frac{\partial (h \circ g \circ f )(x)} {\partial x}  
$$
en $x=7$.

Forward pass:
1. Calcul et stockage de $y=f(x)$
2. Calcul et stockage de $z=g(y)$
3. Calcul et stockage de $t=h(z)$

Backward pass:
1. Calcul de $\frac{\partial t}{\partial z} = h'(z)$
2. Calcul de $\frac{\partial t}{\partial y} = g'(y) h'(z)$.
3. Calcul de $\frac{\partial t}{\partial x} = f'(x) g'(y) h'(z)$.

Faisons cela en tensorflow:

In [4]:
x=tf.Variable(7.)

#forward
with tf.GradientTape(persistent=True) as tape:
    y = tf.cos(x)
    z = 4*y**2
    t = tf.tanh(z)

#backward
print(tape.gradient(t,z).numpy())
print(tape.gradient(t,y).numpy())
print(tape.gradient(t,x).numpy())

0.041513324
0.2503759
-0.1644936


Bien entendu, on n'est pas obligé d'indiquer toutes les étapes:

In [5]:
x=tf.Variable(7.)
with tf.GradientTape(persistent=False) as tape:
    t = tf.tanh(4*tf.cos(x)**2)

print(tape.gradient(t,x).numpy())

-0.1644936


Remarque: comme je ne calcule qu'un seul gradient avec ma tape, je peux mettre `persistent=False`. La mémoire prise par la tape sera libérée plus vite.

***À vous:***  Considérons des scalaires $a,b,c,d$ et les fonctions affines

* $f(x) = ax+b $ et
* $g(x) = cx + d $.


Calculez explicitement $g\circ f(x+\epsilon) - g\circ f(x)$.    


En comparant cet exo et la 'chain rule', vous comprendrez que : les accroissements infinitésimaux des fonctions lisses, se composent de la même manière que les accroissements des fonctions affines.  En bref : toute fonction lisse est localement une fonction affine.


### Régle de l'accumulation

Quand une fonction a plusieurs variables, $g(a,b,...)$, ses dérivées partielles  se calculent sans difficulté. Par ex, pour calculer  $\frac{\partial g(a,b,...)}{\partial a}$ il suffit de considérer uniquement la fonction $a\to g(a,b,...)$.


Par contre, quand  une variable $x$ intervient plusieurs fois:
$$
z=h(x) =  g  [ f_1 (x), f_2 (x) , ...] = g [ y_1,y_2,...]
$$
 Graphe de calcul (dit en diamant):
$$
x \xrightarrow f  \begin{bmatrix}  y_1 \\  y_2 \\ \vdots \end{bmatrix}  \xrightarrow g z
 $$
 Les accroissements s'additionnent (s'accumulent):
$$
\frac {\partial  z  }{\partial x}  =    \sum_i        \frac{\partial y_i }{\partial x}      \frac {\partial z }{\partial y_i} =    \sum_i     f'_i(x) g'(y_i)  
$$


***À vous:*** vous connaissez par cœur la régle de dérivation d'un produit:
$$
(f_1 * f_2)' = f'_1 f_2 + f_1 f'_2
$$
Vérifiez qu'il s'agit d'un cas particulier de la régle d'accumulation. Pour vous aider, considérer le graphe de calcul en diamant:
$$
x \xrightarrow f  \begin{bmatrix}  f_1(x) \\  f_2(x)  \end{bmatrix}  \xrightarrow * f_1(x) * f_2(x)
 $$







Vérifions la régle de l'accumulation en tensorflow:

In [6]:
x=tf.Variable(7.)

with tf.GradientTape(persistent=True) as tape:
    y1=x**2
    y2=tf.cos(x)
    y3=tf.atan(x)
    z=y1*y2/y3

print(tape.gradient(z,x).numpy())

dz_dy1=tape.gradient(z,y1)
dz_dy2=tape.gradient(z,y2)
dz_dy3=tape.gradient(z,y3)

dy1_dx=tape.gradient(y1,x)
dy2_dx=tape.gradient(y2,x)
dy3_dx=tape.gradient(y3,x)

dz_dx = dz_dy1*dy1_dx + dz_dy2*dy2_dx + dz_dy3*dy3_dx
print(dz_dx.numpy())

-15.504779
-15.504779


### Comparaison avec le calcul formel "symbolique"

In [7]:
def fonction_complexe(x,y,z):
    a=atan(x/y)
    b=cos(z**2-x)
    return x*y*a*b/z+a-b

In [8]:
%%time
from tensorflow import cos,atan

x=tf.Variable(7.)
y=tf.Variable(5.)
z=tf.Variable(2.)
with tf.GradientTape() as tape:
    f=fonction_complexe(x,y,z)

[df_dx,df_dy,df_dz]=tape.gradient(f,[x,y,z])
print(df_dx.numpy())
print(df_dy.numpy())
print(df_dz.numpy())

-5.6619678
-1.7493755
17.059452
CPU times: user 14.3 ms, sys: 1.25 ms, total: 15.5 ms
Wall time: 11.7 ms


In [9]:
%%time
import sympy
from sympy import cos,atan

x,y,z=sympy.symbols('x y z')
f=fonction_complexe(x,y,z)
df_dx=sympy.Derivative(f, x).doit()
df_dy=sympy.Derivative(f, y).doit()
df_dz=sympy.Derivative(f, z).doit()

subs={x:7.,y:5.,z:2.}
print(df_dx.evalf(subs=subs))
print(df_dy.evalf(subs=subs))
print(df_dz.evalf(subs=subs))

-5.66196787253764
-1.74937550466067
17.0594520169513
CPU times: user 867 ms, sys: 176 ms, total: 1.04 s
Wall time: 1.19 s


Regardons les expressions que doit retenir `sympy`

In [10]:
print(df_dx)

-x*y*sin(x - z**2)*atan(x/y)/z + x*cos(x - z**2)/(z*(x**2/y**2 + 1)) + y*cos(x - z**2)*atan(x/y)/z + sin(x - z**2) + 1/(y*(x**2/y**2 + 1))


## La technique torch

In [11]:
import torch

### Premier exemple de graph de calcul

Voici un graphe de calcul très simple. Il faut le lire de bas en haut. Les `leaf` sont  `x` et `a`

    x     a
     \   /
       y=x*a
       |
       z=y**2

***A vous:*** Est-il possible que dans un graph de calcul il y ait un cycle?

Il faut préciser `requires_grad=True` sur les `leaf` par rapport auxquelles on veut dériver.

In [12]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.) #par défaut, requires_grad=False
y=a*x
z=y**2
z.backward()

print(f"x.grad:{x.grad}, a.grad:{a.grad}")

x.grad:8.0, a.grad:None


Si on veut aussi dériver par rapport à `a`:

In [13]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.,requires_grad=True)
y=a*x
z=y**2
z.backward()

print(f"x.grad:{x.grad}, a.grad:{a.grad}")

x.grad:8.0, a.grad:4.0


### Second exemple

***A retenir:***

* À tout calcul, on peut mentalement associer un graphe de calcul
* Mais ce graphe est physiquement créé par torch uniquement dans les branches  qui ont la propriété `requires_grad=True`
* Quand on appelle la méthode `backward()` sur un nœud, torch va remonter le graphe depuis ce nœud pour calculer les gradients.


Suivons l'exemple du calcul de dérivée de
$$
z=(x^2*cos(x))^2
$$
Le graph des calcul inclus un diamant puisque $x$ intervient deux fois.

Forward



        x=𝜋
       /   \
    a=x^2   b=cos(x)
     =𝜋²    =-1
     \      /
       y=a*b
        =-𝜋²
        |
       z=y**2
        =𝜋⁴

On déclenche le passage backward par `z.backward()`



Etape 1


    dz/dy=2y
         =-2𝜋²


            





Etape 2
       

      dz/da           dz/db
      =dz/dy*dy/da    =dz/dy*dy/db
      =-2𝜋² *b        =-2𝜋² *a
      =2𝜋²            =-2𝜋⁴
        \            /  
           dz/dy=-2𝜋²

Etape 3

            dz/dx
            =  dz/da*da/dx
              +dz/db*db/dx
            =  2𝜋²* 2x
              -2𝜋⁴* (-sin(x))
            = 4𝜋³
          /          \

      dz/da           dz/db
      =2𝜋²            =-2𝜋⁴
        \            /  
           dz/dy=-2𝜋²

### Dériver par rapport à une variable non-leaf

In [14]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.,requires_grad=True) #par défaut, requires_grad=False
y=a*x
z=y**2

Dans le graph de calcul précédent, la variable `y` n'est pas une `leaf` mais elle `requires_grad`

In [15]:
y.requires_grad, y.is_leaf

(True, False)

Par défaut, on ne peut pas calculer ${\partial z\over \partial y}$, sauf si demande à $y$ de retenir le gradient qui la traverse pendant le backward:

In [16]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.,requires_grad=True) #par défaut, requires_grad=False
y=a*x
y.retain_grad()
z=y**2
z.backward()

print(f"x.grad:{x.grad}, a.grad:{a.grad}, y.grad:{y.grad}")

x.grad:8.0, a.grad:4.0, y.grad:4.0


Il faut vraiment imaginer que les tenseurs torch ne sont pas des nombres, mais des résultats d'évaluation de fonctions composées, et que toutes les étapes du calcul fonctionnel sont mémorisées dans le tenseur.



### Des sources non-scalaire

In [17]:
#un exemple qui ressemble au calcul d'une loss de modèle linéaire

w = torch.ones([2, 3], requires_grad=True)
b = torch.ones([3],requires_grad=True)

x = torch.ones([2])

#début du graph de calcul
y = x@w+b
z = y**2

z.sum().backward()
print("dz/dw:\n",w.grad)
print("dz/db:\n",b.grad)

dz/dw:
 tensor([[6., 6., 6.],
        [6., 6., 6.]])
dz/db:
 tensor([6., 6., 6.])


 Remarquons que `w` et `w.grad` ont les mêmes shape. Logique puisque dériver par rapport à un tenseur signifie simplement dériver par rapport à tous les éléments du tenseur.

***Attention:*** La méthode `y.backward()` nécessite que `y` est un scalaire. Remplacez

    z.sum().backward()

par

    z.backward()

Pour voir le message d'erreur qui apparait.

### Backpopager plusieurs fois à travers un même graphe de calcul.


La méthode `.backward()` provoque  la backpropagation, et lors de cette opération certaines des données stockées dans le graphe sont détruites (pour libérer de l'espace mémoire). Pour éviter cela, on peut utiliser `.backward(retain_graph=True)`




In [18]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2) #par défaut, requires_grad=False
y=a*x
y.backward(retain_graph=True) #tester en supprimant retain_graph=True

z=y**2
z.backward()

print(x.grad)

tensor(10.)


Comme vous pouvez le constater,  les gradients créés par les 2 back-propagations se sont sommés. Si ce n'est pas le résultat souhaité, il faut penser à intercaler la méthode `.zero_()`

In [19]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2) #par défaut, requires_grad=False
y=a*x
y.backward(retain_graph=True) #tester en supprimant retain_graph=True
print(x.grad)
x.grad.zero_()


z=y**2
z.backward()

print(x.grad)

tensor(2.)
tensor(8.)


Ci-dessous, on construit 2 fois le même graph du calcul. Pas besoin de `retain_graph=True` puisqu'on refait les calculs.

Mais ces 2 graphs sont construits à partir de la même variable source `x`. Du coup les gradients des 2 back-propagations s'accumulent dans `x.grad`

In [20]:
x = torch.tensor(1., requires_grad=True)
y = x**2

y.backward()
print(x.grad)

y = x**2 #nouveau graph (on aurait pu changer de nom: z= ...)
y.backward()
print(x.grad)

tensor(2.)
tensor(4.)


### Désactiver les gradients

La méthode `x.requires_grad_(True/False)` permet de changer le status du tenseur à tout moment. Elle finit par un underscore puisqu'elle est `inplace`.


On peut aussi désactiver l'enregistrement des calculs  avec:

        with torch.no_grad():
            calculs...


Il existe plusieurs raisons de désactiver le suivi de gradient :

* Pour marquer certains paramètres de votre réseau neuronal comme paramètres gelés (ils ne sont plus modifiés par les optimizers car ils ne produisent plus de gradients).

* Pour accélérer les calculs lorsque vous effectuez uniquement des passages forward: comparez les temps de calculs des programmes ci-dessous (qui donnent le même résultat).

### Calculer l'occupation de la mémoire gpu

In [21]:
import torch

In [22]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

AttributeError: module 'torch._C' has no attribute '_cuda_resetPeakMemoryStats'

Vérifiez que vous êtes bien à 0 ci-dessus. Sinon cela signifie que vous avez avant construits des tenseurs dans le gpu.

In [ ]:
size=1024

In [ ]:
A=torch.ones(size,device="cuda")
torch.cuda.max_memory_allocated()

In [ ]:
size*4

In [ ]:
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

In [ ]:
A=torch.ones(size,dtype=torch.float64,device="cuda")
torch.cuda.max_memory_allocated()

In [ ]:
size*8

In [ ]:
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

***À vous:*** Que se passe-t-il si l'on remplace 1024 par une taille légèrement plus grande, ou plus petite ? Vous en déduirez pourquoi on aime bien définir des tenseurs dont les tailles sont des puissances de 2.

***À vous:*** Que vérifie-t-on dans la suite ?

In [ ]:
def some_calculus(requires_grad,n):
    A=torch.rand(1000,device="cuda",requires_grad=requires_grad)
    for _ in range(n):
        A=A*torch.rand(1000,device="cuda")
    return A

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(True,10)
print(torch.cuda.max_memory_allocated())
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(True,100)
print(torch.cuda.max_memory_allocated())
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(False,10)
print(torch.cuda.max_memory_allocated())
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(False,100)
print(torch.cuda.max_memory_allocated())
del A

## La fonction grad

### Pour spécifier les sources

Dans la méthode `backward()` on peut spécifier les sources:

In [ ]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.,requires_grad=True) #par défaut, requires_grad=False
y=a*x
z=y**2
z.backward(inputs=[x])

print(f"x.grad:{x.grad}, a.grad:{a.grad}")

Cependant, dans ce cas, c'est plus naturelle d'utiliser la fonction `grad`:

In [ ]:
from torch.autograd import grad

x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.,requires_grad=True) #par défaut, requires_grad=False
y=a*x
z=y**2


print(f"x.grad:{grad(z,x)[0]}")

In [ ]:
print(x.grad)#rien dedans

Mais tout ce qu'on a dit avant reste valable: par exemple, on ne peut pas enchainer le programme précédent par `grad(z,a)` car cela impliquerait une seconde backpropagation dans le graphe des calculs.

Pour pouvoir faire cela, il faut ajouter une option:

In [ ]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.,requires_grad=True) #par défaut, requires_grad=False
y=a*x
z=y**2


print(f"x.grad:{grad(z,x,retain_graph=True)[0]}")
print(f"a.grad:{grad(z,a)[0]}")

Mais ce n'est pas efficace: il vaut mieux calculer tous les gradients d'un coup:

In [ ]:
x=torch.tensor(1.,requires_grad=True)
a=torch.tensor(2.,requires_grad=True) #par défaut, requires_grad=False
y=a*x
z=y**2


print(f"x.grad:{grad(z,[x,a])}")

Ce qui revient exactement au même que de faire un `z.backward()`

### Pour calculer une dérivée seconde

In [ ]:
x=torch.tensor(10.,requires_grad=True)
y=torch.tensor(12.,requires_grad=True)

u=x**2+y
u_x=grad(u,x,create_graph=True)[0]

u_xx=grad(u_x,x)[0]

u_xx.item()

La fonction `grad` contient aussi une option `retain_grad` qui permet d'utiliser plusieurs fois le bout de graph que cette fonction à créer.

### Quelques particularités

Remarquons que la fonction grad renvoie un message d'erreur quand la source n'est pas relié à la cible. Sauf si l'on met l'option `allow_unused=True`

In [ ]:
x=torch.tensor(5.,requires_grad=True)
y=torch.tensor(6.,requires_grad=True)
z=y**2

grad(z,x,allow_unused=True)

Pour avoir 0 (comme on ferait en math) et pas 'None', on fait:

In [ ]:
x=torch.tensor(5.,requires_grad=True)
y=torch.tensor(6.,requires_grad=True)
z=y**2

grad(z,x,allow_unused=True,materialize_grads=True)

Attention quand on dérive plusieurs fois un polynôme: quand la dérivée s'annule, ça plante:

In [ ]:
x=torch.tensor([5.,6],requires_grad=True)
u=3*x**2
u_x=grad(u.sum(),x,create_graph=True)[0]
u_x

In [ ]:
u_xx=grad(u_x.sum(),x,create_graph=True)[0]
u_xx

In [ ]:
u_xxx=grad(u_xx.sum(),x,create_graph=True)[0]
u_xxx

L'erreur ci-dessous vient du résultat ci-dessus: pour le tenseur constant 0, torch a oublié de mettre une `gard_fn`. La dérivation ne peut plus continuer ...

In [ ]:
try:
    u_xxxx=grad(u_xxx.sum(),x,create_graph=True)[0]
    print(u_xxxx)
except Exception as e:
    print(e)

## Plongée profonde dans torch

Je vous conseille vivement de regarder la vidéo suivante qui décortique la technique de différentiation automatique de torch.

[vidéo expliquant la mécanique des tenseurs en torch](https://www.youtube.com/watch?v=MswxJw-8PvE)

## Le jeu des ratages de gradient


***À vous:*** Voici plusieurs programmes où le calcul du gradient échoue. Trouvez l'explication. Debuguez quand c'est possible.

In [ ]:
import torch

### A

In [ ]:
x = torch.tensor(2.,requires_grad=True)
y = torch.tensor(3.,requires_grad=True)

z = y * y
z.backward()
print(x.grad)

#### ♡♡

Explication:



Pas de debug possible, c'est un problème de conception du programme.

### B

In [ ]:
try:
    x = torch.tensor(2.,requires_grad=True)
    y = x**2+1
    z1 = y**3
    z2 = (y-3)**3
    z1.backward()
    z2.backward()
except Exception as e:
    print(e)

#### ♡♡

In [ ]:
DEBUG


    x.grad:348.0

### C

Ci-dessous, on appelle deux fois la méthode `backward()`, et pourtant pas d'erreur pourquoi ?

In [ ]:
y = torch.tensor(2.,requires_grad=True)
z1 = y**3
z2 = (y-3)**3
z1.backward()
z2.backward()

#### ♡♡

### D

l'erreur de base:

In [ ]:
try:
    x = torch.tensor(2.)
    y = x**2+1
    y.backward()
    x.requires_grad_(True)
    print(x.grad)
except Exception as e:
    print(e)

#### ♡♡

In [ ]:
Debug


    tensor(4.)

### E

In [ ]:
x = torch.tensor(2.,requires_grad=True)
#on veut calculer de gradient de y par rapport à x=2, puis x=3, puis ...
for epoch in range(3):
    y = x**2+1
    y.backward()
    print(x.grad)
    #on augmente la valeur de x
    x = x + 1

#### ♡♡

In [ ]:
DEBUG


    tensor(4.)
    tensor(6.)
    tensor(8.)

### F



In [ ]:
try:
    a=torch.tensor(2,requires_grad=True)
    b=a**3
    b.backward()
    print(f"a.grad:{a.grad}")
except Exception as e:
    print(e)

#### ♡♡

In [ ]:
 DEBUG


    a.grad:12.0

### G

In [ ]:
try:
    A=torch.ones([2,2],requires_grad=True)
    B=A**2
    B.backward()
    print(A.grad)
except Exception as e:
    print(e)

#### ♡♡

In [ ]:
DEBUG


## Petits exos



### Dessin

#### ♡♡♡

Déssinez le graph de calcul suivant

In [ ]:
x1 = torch.tensor(2.)
x2 = torch.tensor(0.)
x3 = torch.tensor(1.)
y = (x1+x2)**2
z = torch.exp(y*x3)
z

    tensor(54.5981)

### Des calculs à la main puis à la machine

Considérons
$$
z=y^2
$$
où
$$
y = {x_1 \over x_2}
$$
avec $x_1=2$ et $x_2=1$.

---

On a:
$$
{\partial z \over \partial y} = 2y \text{ en } y={x1\over x2} =2
$$
Donc
$$
{\partial z \over \partial y} =4
$$

---
On a:
$$
{\partial y \over \partial x_2} = -{x_1\over x_2^2} = -2
$$



---
On a:
$$
{\partial z \over \partial x_2} = ...
$$

#### ♡♡

***À vous:*** calculez ces dérivées avec torch en ne créant qu'un seul graph de calcul.